We need to arange the validation data similar to that of test data for the calibration model.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import xml
import xml.etree.ElementTree as ET
import cv2
import math
import warnings
from skimage.transform import resize
import os
from scipy import ndimage
import shutil

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
def create_subwindows_and_counts(image, numpy_folder, stride = 8, kernel_size = 32):
    im_name = image.split(".")[0]
    im_file = im_name + '.npy'
    count_file = im_name + '_density_map.npy'
    # load the image and the count numpy files
    loaded_im_file = np.load(numpy_folder + '/' + im_file)
    loaded_count_file = np.load(numpy_folder + '/' + count_file)
        
    # create the subwindows and counts as follows
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]
    
    density_sums = []
    catch_image = []
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            density = loaded_count_file[i: i + kernel_size, j : j + kernel_size]
            dense_sum = np.sum(density)
            density_sums.append(dense_sum)
            sub_window = resize(sub_window, (32, 32,3))
            catch_image.append(sub_window)

    # save the combined subwindows and counts
    return(catch_image,density_sums, im_file)

In [4]:
# we have the density files and the image files as npy files for validation data stored at Preprocessed_valid_data/all_img_density_files/
# let's use those and create the stackers. 

In [5]:
# image and desnity file location for validation data
valid_file_location = 'Preprocessed_valid_data/all_img_density_files'

In [6]:
# contents at this location, and there shapes?
all_valid_files = os.listdir(valid_file_location)
all_valid_files.sort()

In [7]:
for file in all_valid_files:
    loaded_file = np.load(os.path.join(valid_file_location, file))
    print(loaded_file.shape)

(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)


In [8]:
# get only the images and not the maps
valid_im_names = [item for item in all_valid_files if item.split(".")[0][-3:] != 'map']

In [9]:
# check the count
len(valid_im_names)

32

In [10]:
%%time
# get the subwindows and counts for test data

im_names = []
for image in valid_im_names:
    val_ims, val_maps, val_names = create_subwindows_and_counts(image, valid_file_location, stride = 8, kernel_size = 32)
    val_im_array = np.array(val_ims)
    val_map_array = np.array(val_maps)
    print(image, val_im_array.shape, val_map_array.shape)
    # save the arrays
    # image stack
    save_path = 'final_valid_sub_windows_and_counts/Block_24'
    np.save(save_path + '/' + 'val_ims_' + image, val_im_array)
    # count stack
    np.save(save_path + '/' + 'val_counts_' + image, val_im_array)
    im_names.append(val_names)

Block0204_2020_06_29.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_01.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_02.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_06.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_07.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_08.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_14.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_16.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_17.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_20.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_22.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_23.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_03.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_04.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_06.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_07.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_11.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_12.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_14.npy (12288, 32, 32, 3) (1